In [76]:
from bs4 import BeautifulSoup
from requests import get
import random
import string
import sqlite3
import json
import html.parser as htmlparser
parser = htmlparser.HTMLParser()
import logging

# Input
# flickfilosopher_file = '../Scraping/flickfilosopher.txt'
# current_reviews = set(line.strip() for line in open(flickfilosopher_file))

# Output
# log_path = '../Logs/movieReviews.log'
db_path = '../Data/test.db'
db_name = 'test'
conn = sqlite3.connect(db_path)
c = conn.cursor()

# logger = logging.getLogger()
# fhandler = logging.FileHandler(filename=log_path, mode='a')
# formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
# fhandler.setFormatter(formatter)
# logger.addHandler(fhandler)
# logger.setLevel(logging.DEBUG)

def generateNode(length):
    letters_and_digits = string.ascii_letters + string.digits
    result_str = ''.join((random.choice(letters_and_digits) for i in range(length)))
    node = "_:znode" + result_str
    return node

review_count = 0
no_annotation = 0
no_rating = 0
link_count = 0

host = "https://www.empireonline.com"
review_count = 0
no_annotation = 0

for page in range (1,419):
    url = f'https://www.empireonline.com/movies/reviews/{page}/'
    soup = BeautifulSoup(get(url).text, "lxml")
    movie_container = soup.find_all("div", class_="image-content")
    for link in movie_container:
        review_link = host + link.a['href']
        response = get(review_link)
        review_soup = BeautifulSoup(response.text, 'lxml')
        try:
            data = json.loads("".join(review_soup.find("script", {"type": "application/ld+json"}).contents),strict=False)
            if not data['@type'] == "review":
                no_annotation += 1
                pass
            else:
                reviewBody = data['description'].replace('\n','')
                worstRating = data['reviewRating']['worstRating']
                bestRating = data['reviewRating']['bestRating']
                ratingValue = data['reviewRating']['ratingValue']
                reviewRating = "already included"
                node = generateNode(31)
                # c.execute(f"INSERT OR IGNORE INTO {db_name} (NODE, URL, REVIEWBODY, RATING, REVIEWRATING, BESTRATING, WORSTRATING) VALUES (?,?,?,?,?,?,?);", (node, review_link, reviewBody, str(reviewRating), ratingValue, bestRating, worstRating))
                # conn.commit()
                review_count += 1
        except Exception as e:
            logging.debug("Error: " + str(review_link))
            logging.debug(str(review_soup))
            raise(e)

print()


node:_:znodeNqI8k29TnBq2GSC2dc7rhORcAtd2kg3
url: https://www.empireonline.com/movies/reviews/mandalorian-peyton-reed-hints-s2-director/
reviewBody: If you&amp;#39;re a director with a big, Star Wars-focused announcement to make, there are few better days than May the fourth. Which explains why both Robert Rodriguez and Ant-Man And The Wasp&amp;#39;s ...
worstRating: 1
bestRating: 5
ratingValue: 
